In [1]:
import sys
import os
os.chdir("../")

In [23]:
import numpy as np
import pandas as pd
import data
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
train_df = data.train_df('small')

In [4]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0


In [4]:
df = train_df.sort_values(['user_id','session_id','timestamp','step']).reset_index()

In [9]:
df

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,27774,0004IOZI7CKF,0146f7cb014ba,1541266717,1,search for destination,"Valencia, Spain",DE,"Valencia, Spain",tablet,NaN,NaN,NaN,1.0
1,27775,0004IOZI7CKF,0146f7cb014ba,1541266769,2,interaction item info,3381482,DE,"Valencia, Spain",tablet,NaN,NaN,NaN,1.0
2,27776,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,NaN,110976|88759|150904|3549934|4084196|2094980|11...,70|83|150|143|86|74|78|96|55|72|117|84|92|56|1...,1.0
3,27777,0004IOZI7CKF,0146f7cb014ba,1541266841,4,clickout item,7822344,DE,"Valencia, Spain",tablet,NaN,110976|88759|150904|3549934|4084196|2094980|11...,70|83|150|143|86|74|78|96|55|72|117|84|92|56|1...,1.0
4,27778,0004IOZI7CKF,0146f7cb014ba,1541266868,5,clickout item,110985,DE,"Valencia, Spain",tablet,NaN,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,1.0
5,464545,0008B0X0HC39,1b703eda68d85,1541445203,1,clickout item,2762974,US,"Coron, Philippines",desktop,NaN,2762974,494,1.0
6,696997,000ZN9219G41,1086064207e4f,1541378764,1,filter selection,Air Conditioning,CA,"Lisbon, Portugal",mobile,Air Conditioning,NaN,NaN,1.0
7,696998,000ZN9219G41,1086064207e4f,1541378766,2,filter selection,Car Park,CA,"Lisbon, Portugal",mobile,Air Conditioning|Car Park,NaN,NaN,1.0
8,696999,000ZN9219G41,1086064207e4f,1541378776,3,change of sort order,interaction sort button,CA,"Lisbon, Portugal",mobile,NaN,NaN,NaN,1.0
9,697000,000ZN9219G41,1086064207e4f,1541378786,4,change of sort order,price only,CA,"Lisbon, Portugal",mobile,NaN,NaN,NaN,1.0


In [28]:
# find the last clickout rows
last_clickout_idxs = find_last_clickout_indices(df)
clickout_rows = df.loc[last_clickout_idxs, ['user_id','session_id','action_type','impressions','prices']]
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|').apply(lambda x: list(map(int, x)))
clickout_rows['price_list'] = clickout_rows.prices.str.split('|').apply(lambda x: list(map(int, x)))

In [29]:
clickout_rows

,user_id,session_id,action_type,impressions,prices,impression_list,price_list
4,0004IOZI7CKF,0146f7cb014ba,clickout item,2632452|5747484|110985|84220|3752262|85103|852...,75|62|113|91|106|115|75|68|81|86|36|66|86|159|...,"[2632452, 5747484, 110985, 84220, 3752262, 851...","[75, 62, 113, 91, 106, 115, 75, 68, 81, 86, 36..."
5,0008B0X0HC39,1b703eda68d85,clickout item,2762974,494,[2762974],[494]
16,000ZN9219G41,1086064207e4f,clickout item,149015|1077638|34819|4384770|41629|5525034|348...,52|60|64|66|70|70|71|72|75|76|77|80|80|84|84|8...,"[149015, 1077638, 34819, 4384770, 41629, 55250...","[52, 60, 64, 66, 70, 70, 71, 72, 75, 76, 77, 8..."
20,001OCXQ9PXET,362dc2be48010,clickout item,2450306|1473305|6398796|1161413|13435|231736|1...,67|84|65|79|103|74|76|98|73|98|62|55|121|74|92...,"[2450306, 1473305, 6398796, 1161413, 13435, 23...","[67, 84, 65, 79, 103, 74, 76, 98, 73, 98, 62, ..."
21,001TEVEVUEBE,394f5ad9aa596,clickout item,3366614|2670592|6625742|2881004|5708270|841560...,61|80|102|87|126|85|26|81|50,"[3366614, 2670592, 6625742, 2881004, 5708270, ...","[61, 80, 102, 87, 126, 85, 26, 81, 50]"
26,002IXCPKK20W,dcdc75a82f142,clickout item,2730178|2829436|2551186|9772462|5092274|969192...,84|71|79|30|34|45|68|273|62|35,"[2730178, 2829436, 2551186, 9772462, 5092274, ...","[84, 71, 79, 30, 34, 45, 68, 273, 62, 35]"
28,002O8H8GYKSQ,ab252cd2bafd3,clickout item,3469158|3392402|1394202|107597|4085894|106755|...,136|92|92|74|90|100|73|93|145|68|114|123|148|1...,"[3469158, 3392402, 1394202, 107597, 4085894, 1...","[136, 92, 92, 74, 90, 100, 73, 93, 145, 68, 11..."
29,002Z7VZFWM79,d4cc248f60752,clickout item,56542|79374|3132582|72594|68177|72865|63704|62...,140|114|52|86|140|67|141|53|87|60|38|49|61|43|...,"[56542, 79374, 3132582, 72594, 68177, 72865, 6...","[140, 114, 52, 86, 140, 67, 141, 53, 87, 60, 3..."
40,003728BB2ALE,eaf4b65f13e8e,clickout item,10875780|8784374|3758182|3766642|5058396|99548...,17|31|43|107|41|22|30|36|25|24|36|36|28|27|31|...,"[10875780, 8784374, 3758182, 3766642, 5058396,...","[17, 31, 43, 107, 41, 22, 30, 36, 25, 24, 36, ..."
41,003CQCD8POV7,4c2ae42150f97,clickout item,3139558|2499058|7252818|8117350|3994174|997034...,56|120|102|108|157|50|117,"[3139558, 2499058, 7252818, 8117350, 3994174, ...","[56, 120, 102, 108, 157, 50, 117]"


In [102]:
# find the interactions with numeric reference
reference_rows = df[['user_id','session_id','reference','action_type', 'index']]
reference_rows = reference_rows[df.reference.str.isnumeric() == True].astype({'reference':'int'})
# skip last clickouts
reference_rows = reference_rows.loc[~reference_rows.index.isin(last_clickout_idxs)]
reference_rows = reference_rows.drop('action_type',axis=1)

price_series = np.ones(reference_rows.shape[0], dtype=float) * (-1)

In [103]:
reference_rows

,user_id,session_id,reference,index
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777
17,001OCXQ9PXET,362dc2be48010,13857,884525
18,001OCXQ9PXET,362dc2be48010,13857,884526
23,002F746OAPLZ,5958b15f2af92,68760,618712
24,002F746OAPLZ,5958b15f2af92,68760,618713
25,002IXCPKK20W,dcdc75a82f142,2813356,450142
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439
32,003728BB2ALE,b55afcb9b5e96,2516686,618717


In [104]:
# iterate over the sorted reference_rows and clickout_rows
min_price = 999999
max_price = -999999

j = 0
clickout_indices = clickout_rows.index.values
ckidx = clickout_indices[j]
next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']
k = 0
for row in tqdm(zip(reference_rows.index, reference_rows.user_id, reference_rows.session_id, 
                    reference_rows.reference)):
    idx = row[0]
    # if the current index is over the last clickout, break
    if idx >= clickout_indices[-1]:
        break
    # find the next clickout index
    while idx > clickout_indices[j]:
        j += 1
        ckidx = clickout_indices[j]
        next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
        next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']

    # check if row and next_clickout are in the same session
    if row[1] == next_clickout_user_id and row[2] == next_clickout_sess_id:
        impress = clickout_rows.at[ckidx, 'impression_list']
        row_reference = row[3]
        if row_reference in impress:
            ref_idx = impress.index(row_reference)
            price_list = clickout_rows.at[ckidx, 'price_list']
            ref_price = price_list[ref_idx]
            price_series[k] = ref_price
            # update min and max
            min_price = min(min_price, min(price_list))
            max_price = max(max_price, max(price_list))
            
    k += 1

print(min_price, max_price)

5 9996


In [105]:
min_price = np.log(min_price +1)
max_price = np.log(max_price +1)
print(min_price, max_price)

1.791759469228055 9.210040326967182


In [107]:
reference_rows['price'] = price_series
reference_rows

,user_id,session_id,reference,index,price
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775,-1.0
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776,-1.0
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777,-1.0
17,001OCXQ9PXET,362dc2be48010,13857,884525,-1.0
18,001OCXQ9PXET,362dc2be48010,13857,884526,-1.0
23,002F746OAPLZ,5958b15f2af92,68760,618712,-1.0
24,002F746OAPLZ,5958b15f2af92,68760,618713,-1.0
25,002IXCPKK20W,dcdc75a82f142,2813356,450142,-1.0
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439,92.0
32,003728BB2ALE,b55afcb9b5e96,2516686,618717,-1.0


In [110]:
# mask not available prices
mask_na = price_series != -1

# log and scaling
price_series[mask_na] = np.log(price_series[mask_na] +1)
min_price = np.log(min_price +1)
max_price = np.log(max_price +1)
# scale min-max
price_series[mask_na] = (price_series[mask_na] - min_price) / (max_price - min_price)

# reset to 0 not available prices
price_series[~mask_na] = 0

In [111]:
reference_rows['price'] = price_series
reference_rows

,user_id,session_id,reference,index,price
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775,0.000000
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776,0.000000
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777,0.000000
17,001OCXQ9PXET,362dc2be48010,13857,884525,0.000000
18,001OCXQ9PXET,362dc2be48010,13857,884526,0.000000
23,002F746OAPLZ,5958b15f2af92,68760,618712,0.000000
24,002F746OAPLZ,5958b15f2af92,68760,618713,0.000000
25,002IXCPKK20W,dcdc75a82f142,2813356,450142,0.000000
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439,2.703732
32,003728BB2ALE,b55afcb9b5e96,2516686,618717,0.000000


In [40]:
df.loc[[27]]

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
27,368439,002O8H8GYKSQ,ab252cd2bafd3,1541328660,1,clickout item,3392402,NL,"Veenendaal, Netherlands",desktop,NaN,3469158|3392402|1394202|107597|4085894|106755|...,136|92|92|74|90|100|73|93|145|68|114|123|148|1...,1.0


## Test the feature

In [112]:
from extract_features.rnn.reference_price_in_last_clickout import ReferencePriceInLastClickout
feat = ReferencePriceInLastClickout()

In [113]:
feat.join_to(train_df)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_price_in_last_clickout feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,0.000000
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,0.000000
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0,0.000000
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0.000000
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0,0.000000
